In [9]:
import pandas as pd
from bertopic import BERTopic

In [10]:
# 1. Load the dataset
df = pd.read_csv("preprocessed_redditData-removedtopic.csv")

In [11]:
# 2. Prepare inputs for BERTopic
texts = df['text'].astype(str).tolist()
timestamps = df['year'].astype(str).tolist()  # Treat year as string labels

In [12]:
# 4. Fit BERTopic model
topic_model = BERTopic(verbose=True)
topics, probs = topic_model.fit_transform(texts)

2025-07-15 15:28:34,191 - BERTopic - Embedding - Transforming documents to embeddings.
Batches: 100%|██████████| 2000/2000 [03:04<00:00, 10.82it/s]
2025-07-15 15:31:43,866 - BERTopic - Embedding - Completed ✓
2025-07-15 15:31:43,867 - BERTopic - Dimensionality - Fitting the dimensionality reduction algorithm
2025-07-15 15:32:04,600 - BERTopic - Dimensionality - Completed ✓
2025-07-15 15:32:04,607 - BERTopic - Cluster - Start clustering the reduced embeddings
2025-07-15 15:32:13,198 - BERTopic - Cluster - Completed ✓
2025-07-15 15:32:13,226 - BERTopic - Representation - Fine-tuning topics using representation models.
2025-07-15 15:32:14,918 - BERTopic - Representation - Completed ✓


In [13]:
# 5. Get topics per year (class)
topics_by_year = topic_model.topics_per_class(
    docs=texts,
    classes=timestamps,
    global_tuning=True
)

6it [00:03,  1.54it/s]


In [14]:
# 6. Extract top 20 topics per year
top20_by_year = {}
for yr in sorted(df['year'].unique()):
    df_year = topics_by_year[topics_by_year['Class'] == str(yr)]
    top20 = df_year.sort_values('Frequency', ascending=False).head(20)
    top20_by_year[yr] = top20

In [15]:
print(topics_by_year.columns)

Index(['Topic', 'Words', 'Frequency', 'Class'], dtype='object')


In [16]:
# 7. Print results 
for yr, df_topics in top20_by_year.items():
    print(f"\nTop 20 topics for {yr}")
    print(df_topics[['Topic', 'Words', 'Frequency']])


Top 20 topics for 2020
      Topic                                              Words  Frequency
1004     -1            csgo, gacha, overwatch, gameplay, would       3757
1006      1                         fps, ram, laptop, gpu, cpu         91
1064     59              beta, close, closed, register, access         79
1008      3  map, geoculuses, pixelsmash, interactable, bac...         65
1013      8          dragon, dragonborn, ender, defeat, fossil         60
1005      0              modding, mod, modders, modder, author         57
1019     14                          euw, na, norm, lf, friend         56
1037     32              meme, effort, meepcity, editnote, sfs         54
1007      2              skin, prestige, trading, mvp, phanton         49
1084     79              cyberpunk, projekt, cd, witcher, cdpr         48
1009      4                   ps5, playstation, ps4, sony, ps3         40
1023     18           house, apartment, creek, willow, mansion         39
1129    124   

In [ ]:
#lda
from gensim import corpora
from gensim.models import LdaModel
from gensim.utils import simple_preprocess